# **라이브러리 및 환경변수 설정**


In [ ]:
import pandas as pd
from langchain_chroma import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from supabase import create_client, Client
from app.routers.vectorstore import request_table, text_files_to_docs, docs_text_split, docs_insert_db, db_to_document
import os
from typing import List


SUPABASE_URL = os.getenv('SUPABASE_URL')
SUPABASE_API_KEY = os.getenv('SUPABASE_API_KEY')

supabase: Client = create_client(supabase_url=SUPABASE_URL, supabase_key=SUPABASE_API_KEY)

def check_vectorstore_status():
    collection = vector_store._collection
    print(f"저장된 문서 수: {collection.count()}")
    
    # 샘플 검색으로 테스트
    docs = retriever.get_relevant_documents("테스트")
    print(f"검색된 문서 수: {len(docs)}")
    for doc in docs:
        print(f"문서 내용: {doc.page_content[:100]}...")

# **신규 데이터 삽입**

In [ ]:
# '.txt' files convert to Document object
text_into_docs = text_files_to_docs(path = './data/GA_information')

# Document object split chunk
splitted_docs = docs_text_split(docs = text_into_docs, size= 500, overlap = 100, db = supabase)

# New Documents insert into db
docs_insert_db(splitted_docs = splitted_docs, db = supabase)

# Load latest docs from the db
latest_docs = db_to_document(db = supabase)

# HuggingFaceModel Alert Disable
os.environ["TOKENIZERS_PARALLELISM"] = "false"

[Alert] No Data in table
[Alert] All data get Success
[Alert] All data get Success


/opt/anaconda3/envs/ga_chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Alert] There is no Chroma DB, Create a new one.


In [4]:
# Define Embedding Model
embedding = HuggingFaceEmbeddings(
    model_name = 'FronyAI/frony-embed-large-ko-v1',
    model_kwargs = {'device': 'mps'},
    encode_kwargs = {'normalize_embeddings': True}
)

# Define Vectorstore path
vectorstore_path = './ga_assistant_store'

# If not exist vectorstore
if not os.path.isdir(vectorstore_path):
    print("[Alert] There is no Chroma DB, Create a new one.")
    vector_store = Chroma.from_documents(
        documents = latest_docs,                    # Document 객체 리스트 (TextSplitter로 분할된 문서)
        embedding = embedding,                      # 임베딩 모델 정의 변수
        collection_name = "ga_assistant",           # 필요할 때 호출할 수 있는 컬렉션 이름 정의
        persist_directory = vectorstore_path        # 로컬 경로 지정 (미지정 시 RAM에 저장 -> 휘발되니 경로 지정 필요)
    )

# If exist vectorstore
else:
    print("[Alert] Discover DB, add data to existing DB.")
    vector_store = Chroma(
        collection_name = "ga_assistant",
        embedding_function = embedding,
        persist_directory = vectorstore_path
    )

    # Add new documents
    # vector_store.add_documents(latest_docs)

/opt/anaconda3/envs/ga_chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Alert] Discover DB, add data to existing DB.


# **Compressor retriever**

In [5]:
retriever = vector_store.as_retriever(search_kwargs = {'k': 3})

CrossEncoder = HuggingFaceCrossEncoder(model_name = 'BAAI/bge-reranker-v2-m3')

re_ranker = CrossEncoderReranker(
    model = CrossEncoder,
    top_n = 2
)

EmbeddingFilter = EmbeddingsFilter(
    embeddings = embedding,
    similarity_threshold = 0.3
)

compressor_pipeline = DocumentCompressorPipeline(
    transformers = [re_ranker, EmbeddingFilter]
)

final_retriever = ContextualCompressionRetriever(
    base_compressor = compressor_pipeline,
    base_retriever = retriever
)

# **평가 데이터셋 생성**

In [12]:
# Load API Key
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# Definition model object
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',         # model name
    google_api_key = GEMINI_API_KEY,    # API KEY
    temperature = 0.3
)

## **Pydantic Class**

In [10]:
class QAPair(BaseModel):
    query: str = Field(description = "AI가 생성한 질문 (write your question in KOREAN)")
    label: str = Field(description = "질문의 대한 답 (write the answer to the fact_based question in KOREAN, making sure it reflects the essence of the question)")

class QASet(BaseModel):
    qa_pairs: List[QAPair] = Field(description = "query: label의 리스트")

pydantic_parser = PydanticOutputParser(pydantic_object = QASet)

pydantic_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"QAPair": {"properties": {"query": {"description": "AI가 생성한 질문 (write your question in KOREAN)", "title": "Query", "type": "string"}, "label": {"description": "질문의 대한 답 (write the answer to the fact_based question in KOREAN, making sure it reflects the essence of the question)", "title": "Label", "type": "string"}}, "required": ["query", "label"], "title": "QAPair", "type": "object"}}, "properties": {"qa_pairs": {"description": "query: label의 리스트", "items": {"$ref": "#/$defs/QAPair"}, "title": "Qa Pairs", "type": "array"}}, 

## **Create Chain**

In [13]:
prompt_text = ""

with open('./prompt/get_evaluation_data.txt', 'r') as f:
    prompt_text = f.read()

create_dataset_prompt = ChatPromptTemplate.from_template(
    template = prompt_text,
    partial_variables = {'format_instructions': pydantic_parser.get_format_instructions()}
)

chain = create_dataset_prompt | llm | pydantic_parser

def create_dataset(context: str, num_questions: int) -> QASet:
    """ Query functions using chains """
    return chain.invoke({'context': context, 'num_questions_per_chunk': num_questions})

## **Create Evaluation DataSet**

In [6]:
df = request_table('vectorstore', db = supabase)
df.drop(['id', 'source'], axis = 1, inplace = True)

df.head()

[Alert] All data get Success


,doc_id,file_name,page_content,date
0,1,Guide_GA.txt,총무팀 업무 메뉴얼을 안내합니다.\n\n[총무팀 구성원]\n1. 이동훈님 (이메일:...,2025-08-22
1,2,Guide_GA.txt,"[업무 요청]\n1. 유형자산, 무형자산 신청\n 1) 현재 우리 회사는 직군 별 ...",2025-08-22
2,3,Guide_GA.txt,2. 정기 주차\n 1) 안내 : 현재 우리 회사는 50대의 무료 주차분이 할당되어...,2025-08-22
3,4,Guide_GA.txt,3. 자산 외부 반출\n 1) 안내 : 지급된 사내 자산에 대해선 보안상 외부 반출...,2025-08-22


In [14]:
qa_pairs = []

for _, row in df.iterrows():
    qa_pairs_row = create_dataset(row['page_content'], 10)
    
    for qa_pair_row in qa_pairs_row.qa_pairs:
        qa_pairs.append(
            {
                'doc_id': row['doc_id'],
                'context': row['page_content'],
                'file_name': row['file_name'],
                'date': row['date'],
                'query': qa_pair_row.query,
                'label': qa_pair_row.label
            }
        )

final_df = pd.DataFrame(qa_pairs)

In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   doc_id     40 non-null     object
 1   context    40 non-null     object
 2   file_name  40 non-null     object
 3   date       40 non-null     object
 4   query      40 non-null     object
 5   label      40 non-null     object
dtypes: object(6)
memory usage: 2.0+ KB


In [16]:
final_df.head()

,doc_id,context,file_name,date,query,label
0,1,총무팀 업무 메뉴얼을 안내합니다.\n\n[총무팀 구성원]\n1. 이동훈님 (이메일:...,Guide_GA.txt,2025-08-22,총무팀 구성원 중 한 명의 이름은 무엇인가요?,이동훈님입니다.
1,1,총무팀 업무 메뉴얼을 안내합니다.\n\n[총무팀 구성원]\n1. 이동훈님 (이메일:...,Guide_GA.txt,2025-08-22,이동훈님의 이메일 주소는 무엇인가요?,main3373@gmail.com입니다.
2,1,총무팀 업무 메뉴얼을 안내합니다.\n\n[총무팀 구성원]\n1. 이동훈님 (이메일:...,Guide_GA.txt,2025-08-22,이동훈님의 연락처는 몇 번인가요?,010-3271-7132입니다.
3,1,총무팀 업무 메뉴얼을 안내합니다.\n\n[총무팀 구성원]\n1. 이동훈님 (이메일:...,Guide_GA.txt,2025-08-22,이동훈님의 역할 중 하나는 무엇인가요?,총무 일괄 지원입니다.
4,1,총무팀 업무 메뉴얼을 안내합니다.\n\n[총무팀 구성원]\n1. 이동훈님 (이메일:...,Guide_GA.txt,2025-08-22,이동훈님은 어떤 IT 관련 지원을 담당하나요?,"PC, 소프트웨어 장애 등 IT 관련 지원을 담당합니다."
